In [1]:
# Transfer Learning 
## Classification Problem - Emergency vs Non-emergency Vehicle Classification

## 1. Import neccessary libraries
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize

from keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer


In [3]:
## 2. Load the data
#reading the csv file containing data labels
data = pd.read_csv('../data/emergency_classification.csv')

print(data.head())


  image_names  emergency_or_not
0       0.jpg                 1
1       1.jpg                 1
2       2.jpg                 1
3       3.jpg                 1
4       4.jpg                 1


In [4]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('../data/images/' + img_name)
    # pile it one over the other
    X.append(img)


In [5]:
# convert this python list to a single numpy array
X = np.array(X)

#getting the labels for images
y = data.emergency_or_not.values
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency ,
#with only one of them is true for every image
y = to_categorical(y)



In [6]:
## 3. Pre-Process Data
'''
Steps : 
1. Pre-process the data as per model's requirement
2. Prepare training and validation set
'''
#show maximum and minimum values for the image array
X.min(), X.max()

print(X.min(), X.max())


0 255


In [7]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X, data_format=None)

#print minimum and maximum values present in the array
X.min(), X.max()

print(X.min(), X.max()
)


-123.68 151.061


In [8]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X, data_format=None)

#print minimum and maximum values present in the array
X.min(), X.max()

print(X.min(), X.max()
)
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load weights of pretrained model
# creating model with pre trained imagenet weights
base_model = VGG16(weights='imagenet')

#shows model summary
base_model.summary()


-233.558 27.381004


2024-02-05 00:09:32.551613: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-05 00:09:32.551721: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-05 00:09:32.551761: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-05 00:09:32.553309: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-05 00:09:32.557231: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
# creating a VGG16 model with imagenet pretrained weights , accepting input of shape (224,224,3)
# also remove the final layers from model(include_top= False)
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

# show model summary
base_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
'''
## 5. Fine tune the model for the current problem
Steps:-
1. Extract features
2. Flatten the data
3. Rescale features
4. Create a Neural Network Model
5. Compile the model
6. Train and Validate the model
'''
# extract features using the pretrained VGG16 model
# for training set
base_model_pred = base_model.predict(X_train)
#for validation set
base_model_pred_valid = base_model.predict(X_valid)

#show shape of predictions
print(base_model_pred.shape)



total_elements = base_model_pred.size  # This should be 41294848 in your case
new_row_count = 1152
new_col_count = total_elements // new_row_count  # Use floor division to get an integer result

base_model_pred = base_model_pred.reshape(new_row_count, new_col_count)



# flattening the model output to one dimension for every sample of training set
base_model_pred = base_model_pred.reshape(1152, 7*7*512)

print(base_model_pred.shape)

print(base_model_pred_valid.shape)

# flattening the model output to one dimension for every sample of validation set
base_model_pred_valid = base_model_pred_valid.reshape(494, 7*7*512)

print(base_model_pred_valid.shape)




2024-02-05 00:10:19.850871: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 25s 1s/step
(1646, 7, 7, 512)


ValueError: cannot reshape array of size 41294848 into shape (1152,25088)

In [ ]:
# checking the min and max of the extracted features
base_model_pred.min(), base_model_pred.max()

#get maximum value from generated features
max_val = base_model_pred.max()

#normalizing features generated from the VGG16 model to [0,1]
base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val
base_model_pred.min(), base_model_pred.max()

#create a sequential model 
model = Sequential()
# add input layer to the model that accepts input of shape 7*7*512
model.add(InputLayer((7*7*512, )))
# add fully connected layer with 1024 neurons and relu activation
model.add(Dense(units=1024, activation='relu'))
# add fully connected layer with 2 neurons and relu activation
model.add(Dense(units=2, activation='softmax'))

# compile the model
model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')

model.summary()


In [ ]:
# train model using features generated from VGG16 model
model.fit(base_model_pred, y_train, epochs=100, validation_data=(base_model_pred_valid, y_valid))

## 6. Get Predictions
# get predictions
predictions = model.predict_classes(base_model_pred_valid)
#show predictions
print(predictions)
